In [ ]:
!pip install librosa matplotlib numpy scipy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
from scipy import signal
from matplotlib.colors import LinearSegmentedColormap

def create_waveform_art(audio_path, output_path='waveform_art.png'):
    """Minimalistic waveform visualization"""
    y, sr = librosa.load(audio_path)

    fig, ax = plt.subplots(figsize=(16, 9), facecolor='black')
    ax.set_facecolor('black')

    # Downsample for cleaner look
    downsample = len(y) // 5000
    y_plot = y[::downsample]
    t = np.linspace(0, len(y)/sr, len(y_plot))

    ax.fill_between(t, y_plot, color='white', alpha=0.8)
    ax.plot(t, y_plot, color='white', linewidth=0.5, alpha=0.3)

    ax.set_xlim(0, len(y)/sr)
    ax.axis('off')
    plt.tight_layout(pad=0)
    plt.savefig(output_path, dpi=300, facecolor='black')
    plt.close()
    print(f"Saved: {output_path}")

def create_frequency_spiral(audio_path, output_path='frequency_spiral.png'):
    """Spiral visualization based on frequency content"""
    y, sr = librosa.load(audio_path)

    # Chromagram for pitch content
    chroma = librosa.feature.chroma_cqt(y=y, sr=sr, hop_length=2048)
    chroma_mean = np.mean(chroma, axis=1)

    fig, ax = plt.subplots(figsize=(12, 12), facecolor='white')
    ax.set_facecolor('white')

    # Create spiral
    theta = np.linspace(0, 8*np.pi, len(chroma.T))
    r = np.linspace(0.1, 1, len(chroma.T))

    for i in range(12):
        intensity = chroma[i, :]
        x = r * np.cos(theta) * (1 + intensity * 0.5)
        y = r * np.sin(theta) * (1 + intensity * 0.5)
        ax.plot(x, y, color='black', linewidth=0.3, alpha=0.6)

    ax.set_xlim(-1.2, 1.2)
    ax.set_ylim(-1.2, 1.2)
    ax.axis('off')
    plt.tight_layout(pad=0)
    plt.savefig(output_path, dpi=300, facecolor='white')
    plt.close()
    print(f"Saved: {output_path}")

def create_spectrogram_art(audio_path, output_path='spectrogram_art.png'):
    """Minimalistic spectrogram with custom colormap"""
    y, sr = librosa.load(audio_path)

    # Focus on recorder range (500-2500 Hz)
    D = librosa.stft(y)
    S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)

    fig, ax = plt.subplots(figsize=(16, 9), facecolor='#1a1a1a')

    # Custom monochrome colormap
    colors = ['#1a1a1a', '#ffffff']
    cmap = LinearSegmentedColormap.from_list('custom', colors)

    img = librosa.display.specshow(S_db, sr=sr, x_axis=None, y_axis=None,
                                    cmap=cmap, ax=ax)

    ax.axis('off')
    plt.tight_layout(pad=0)
    plt.savefig(output_path, dpi=300, facecolor='#1a1a1a')
    plt.close()
    print(f"Saved: {output_path}")

def create_harmonic_lines(audio_path, output_path='harmonic_lines.png'):
    """Vertical lines based on harmonic content"""
    y, sr = librosa.load(audio_path)

    # Separate harmonics and percussive
    y_harm, y_perc = librosa.effects.hpss(y)

    # Get spectral centroid over time
    cent = librosa.feature.spectral_centroid(y=y_harm, sr=sr)[0]
    times = librosa.frames_to_time(np.arange(len(cent)), sr=sr)

    fig, ax = plt.subplots(figsize=(16, 9), facecolor='#f5f5f5')
    ax.set_facecolor('#f5f5f5')

    # Normalize centroid
    cent_norm = (cent - np.min(cent)) / (np.max(cent) - np.min(cent))

    for i, (t, c) in enumerate(zip(times, cent_norm)):
        ax.plot([t, t], [0, c], color='black', linewidth=0.5, alpha=0.7)

    ax.set_xlim(0, times[-1])
    ax.set_ylim(0, 1)
    ax.axis('off')
    plt.tight_layout(pad=0)
    plt.savefig(output_path, dpi=300, facecolor='#f5f5f5')
    plt.close()
    print(f"Saved: {output_path}")

# Example usage
if __name__ == "__main__":
    import os
    from pathlib import Path

    # Directory containing MP3 files
    audio_dir = "/content/"  # Current directory, change to your path

    # Find all MP3 files
    mp3_files = list(Path(audio_dir).glob("*.mp3"))

    if not mp3_files:
        print("No MP3 files found in the directory!")
    else:
        print(f"Found {len(mp3_files)} MP3 file(s)")

        # Create output directory for artwork
        output_dir = Path("artwork")
        output_dir.mkdir(exist_ok=True)

        for mp3_file in mp3_files:
            filename = mp3_file.stem  # filename without extension
            print(f"\nProcessing: {mp3_file.name}")

            try:
                create_waveform_art(
                    str(mp3_file),
                    str(output_dir / f"{filename}_waveform.png")
                )
                create_frequency_spiral(
                    str(mp3_file),
                    str(output_dir / f"{filename}_spiral.png")
                )
                create_spectrogram_art(
                    str(mp3_file),
                    str(output_dir / f"{filename}_spectrogram.png")
                )
                create_harmonic_lines(
                    str(mp3_file),
                    str(output_dir / f"{filename}_lines.png")
                )
                print(f"✓ Completed artwork for {mp3_file.name}")
            except Exception as e:
                print(f"✗ Error processing {mp3_file.name}: {e}")

        print(f"\n{'='*50}")
        print(f"All artwork saved to '{output_dir}/' directory!")
        print(f"{'='*50}")

In [ ]:
import os
import gc
import numpy as np
import matplotlib.pyplot as plt
from pydub import AudioSegment
from scipy import signal
from pathlib import Path

def load_audio(file_path, max_duration=120):
    """Load MP3 file and convert to numpy array (limit duration to save RAM)"""
    audio = AudioSegment.from_mp3(file_path)

    # Limit to max_duration seconds to prevent RAM overflow
    if len(audio) > max_duration * 1000:
        audio = audio[:max_duration * 1000]
        print(f"  Truncated to {max_duration}s to save memory")

    # Convert to mono
    if audio.channels > 1:
        audio = audio.set_channels(1)

    # Downsample if needed (reduces memory significantly)
    if audio.frame_rate > 22050:
        audio = audio.set_frame_rate(22050)

    samples = np.array(audio.get_array_of_samples(), dtype=np.float32)
    sample_rate = audio.frame_rate

    return samples, sample_rate

def compute_spectrogram(samples, sample_rate):
    """Compute frequency spectrum with memory-efficient settings"""
    # Downsample further if still too large
    if len(samples) > 5_000_000:
        factor = len(samples) // 5_000_000 + 1
        samples = samples[::factor]
        sample_rate = sample_rate // factor
        print(f"  Downsampled by factor {factor}")

    frequencies, times, spectrogram = signal.spectrogram(
        samples,
        sample_rate,
        nperseg=1024,  # Reduced from 2048
        noverlap=512,  # Reduced overlap
        nfft=1024
    )

    return frequencies, times, spectrogram

def create_artistic_visualization(directory_path):
    """Process all MP3 files and create artistic frequency visualization"""

    mp3_files = list(Path(directory_path).glob('*.mp3'))

    if not mp3_files:
        print(f"No MP3 files found in {directory_path}")
        return

    print(f"Found {len(mp3_files)} MP3 file(s)\n")

    cmaps = ['magma', 'plasma', 'inferno', 'viridis', 'twilight']

    for idx, mp3_file in enumerate(mp3_files):
        print(f"[{idx+1}/{len(mp3_files)}] Processing: {mp3_file.name}")

        try:
            # Load and process audio
            samples, sample_rate = load_audio(str(mp3_file))
            frequencies, times, spectrogram = compute_spectrogram(samples, sample_rate)

            # Convert to dB scale
            spectrogram_db = 10 * np.log10(spectrogram + 1e-10)

            # Create figure
            fig, ax = plt.subplots(figsize=(12, 5))
            fig.patch.set_facecolor('#0a0a0a')
            ax.set_facecolor('#0a0a0a')

            # Plot spectrogram
            ax.pcolormesh(
                times,
                frequencies,
                spectrogram_db,
                shading='gouraud',
                cmap=cmaps[idx % len(cmaps)],
                vmin=np.percentile(spectrogram_db, 10),
                vmax=np.percentile(spectrogram_db, 95)
            )

            ax.set_ylim([0, 8000])
            ax.set_xlabel('Time (s)', fontsize=11, color='white', fontweight='light')
            ax.set_ylabel('Frequency (Hz)', fontsize=11, color='white', fontweight='light')
            ax.tick_params(colors='white', labelsize=9)
            ax.spines['bottom'].set_color('#333333')
            ax.spines['left'].set_color('#333333')
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            ax.grid(True, alpha=0.1, color='white', linewidth=0.5)

            plt.tight_layout()

            # Save
            output_filename = mp3_file.stem + '_frequency.png'
            output_path = os.path.join(directory_path, output_filename)
            plt.savefig(output_path, dpi=150, facecolor='#0a0a0a', edgecolor='none')
            print(f"  ✓ Saved: {output_filename}\n")

            # Critical: Close figure and clear memory
            plt.close(fig)
            del samples, frequencies, times, spectrogram, spectrogram_db
            gc.collect()

        except Exception as e:
            print(f"  ✗ Error: {e}\n")
            plt.close('all')
            gc.collect()
            continue

    print(f"Complete! All artworks saved to: {directory_path}")

if __name__ == "__main__":
    directory = "/content/"
    create_artistic_visualization(directory)